In [5]:
from time import time
t00 = time()
import numpy as np

In [6]:
# Load dataset
X_fn = './data/X_reference.npy'
y_fn = './data/y_reference.npy'
X = np.load(X_fn)
y = np.load(y_fn)
print(X.shape, y.shape)

(60000, 1000) (60000,)


In [8]:
# CNN parameters
layers = 6
hidden_size = 100
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1000
in_channels = 64
n_classes = 30

In [8]:
import os
import torch
# Set up GPU/CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()

In [6]:
from resnet import ResNet
from datasets import spectral_dataloader
from training import run_epoch
from torch import optim


In [9]:
# Initialize a new model (from scratch)
cnn = ResNet(hidden_sizes, num_blocks, input_dim=input_dim,
             in_channels=in_channels, n_classes=n_classes)

In [12]:
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))

In [13]:
# Define the validation split
p_val = 0.1
n_val = int(len(y) * p_val)  # Adjusting the validation size based on your new dataset size
idx_tr = list(range(len(y))) 
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

In [14]:
# Set up DataLoaders
batch_size = 10
dl_tr = spectral_dataloader(X, y, idxs=idx_tr, batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X, y, idxs=idx_val, batch_size=batch_size, shuffle=False)

In [15]:
# Training parameters
epochs = 30  
best_val = 0
no_improvement = 0
max_no_improvement = 5  # Early stopping criteria
t0 = time()

In [16]:
print('Starting training from scratch!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.2f}s'.format(epoch + 1, time() - t0))
    
    # Train the model
    acc_tr, loss_tr = run_epoch(epoch, cnn, dl_tr, cuda, training=True, optimizer=optimizer)
    print('  Train acc: {:0.2f}, Train loss: {:0.4f}'.format(acc_tr, loss_tr))
    
    # Validate the model
    acc_val, loss_val = run_epoch(epoch, cnn, dl_val, cuda, training=False, optimizer=optimizer)
    print('  Val acc  : {:0.2f}, Val loss  : {:0.4f}'.format(acc_val, loss_val))
    
    # Check for early stopping
    if acc_val > best_val or epoch == 0:
        best_val = acc_val
        no_improvement = 0
        # Save the best model if desired
        torch.save(cnn.state_dict(), 'best_model.pth')
    else:
        no_improvement += 1
    
    if no_improvement >= max_no_improvement:
        print('Early stopping after {} epochs.'.format(epoch + 1))
        break

print('\nTraining completed in: {:0.2f}s'.format(time() - t0))

Starting training from scratch!
 Epoch 1: 0.02s


  Train acc: 83.66, Train loss: 0.0500
  Val acc  : 82.35, Val loss  : 0.0748
 Epoch 2: 1757.28s
  Train acc: 91.59, Train loss: 0.0242
  Val acc  : 90.18, Val loss  : 0.0485
 Epoch 3: 3480.82s
  Train acc: 92.94, Train loss: 0.0199
  Val acc  : 89.63, Val loss  : 0.0616
 Epoch 4: 5256.64s
  Train acc: 93.82, Train loss: 0.0171
  Val acc  : 93.10, Val loss  : 0.0195
 Epoch 5: 7027.02s
  Train acc: 94.49, Train loss: 0.0153
  Val acc  : 90.63, Val loss  : 0.0342
 Epoch 6: 8968.10s
  Train acc: 95.05, Train loss: 0.0139
  Val acc  : 90.55, Val loss  : 0.1389
 Epoch 7: 10893.23s
  Train acc: 95.36, Train loss: 0.0127
  Val acc  : 91.03, Val loss  : 0.0564
 Epoch 8: 12754.06s
  Train acc: 95.80, Train loss: 0.0115
  Val acc  : 89.78, Val loss  : 0.1342
 Epoch 9: 14844.24s
  Train acc: 96.07, Train loss: 0.0108
  Val acc  : 92.15, Val loss  : 0.0298
Early stopping after 9 epochs.

Training completed in: 16778.58s


In [2]:
import torch
import numpy as np

def get_predictions(model, data_loader, cuda=False):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            if cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
                
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return np.array(all_preds), np.array(all_labels)

In [10]:
# Load the trained weights into the model
cnn.load_state_dict(torch.load('best_model.pth'))

# Set the model to evaluation mode
cnn.eval()




ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-